In [1]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('max_seq_item', None)

In [3]:
root = ET.parse('../../data/raw/20220301-susan-merriam/IA_FIRM_SEC_Feed_02_28_2022.xml').getroot()

In [32]:
# Initialize df column lists
dict_items = {}
for firm in root[0]:
    for item in firm:  # Info, MainAddr, etc.
        if item.tag == 'FormInfo':
            for subitem in item:  # Part1A
                for item_n in subitem:  # Item 1, Item 2A, etc.
                    if item_n.tag not in dict_items.keys():
                        dict_items[item_n.tag] = {}
                    for q in item_n.attrib.keys():
                        if q not in dict_items[item_n.tag].keys():
                            dict_items[item_n.tag][q] = []

# ACTUALLY populate df column lists
ls_busnms = []
for firm in root[0][:10]:  # REMEMBER TO REMOVE INDEX UP TO 10 WHEN READY FOR FULL FILE
    for item in firm:  # Info, MainAddr, etc.
        if item.tag == 'Info':
            ls_busnms.append(item.attrib['BusNm'])
        elif item.tag == 'FormInfo':
            for subitem in item:  # Part1A
                for item_n in subitem:  # Item 1, Item 2A, etc.
                    for q, qval in dict_items[item_n.tag].items():
                        if q in item_n.attrib.keys():
                            dict_items[item_n.tag][q].append(item_n.attrib[q])
                        else:
                            dict_items[item_n.tag][q].append(np.nan)

In [69]:
# Unpack the horrible dictionary of dictionaries into a nice list of lists (ha)

ls_col_names = []
ls_cols_lists = []
for item, qdict in dict_items.items():
    for q, ls_q in qdict.items():
        ls_col_names.append(q)
        ls_cols_lists.append(ls_q)
        
ls_final = [ls_busnms] + ls_cols_lists

In [71]:
# Convert list of lists into an inverted dataframe
df_init = pd.DataFrame(ls_final)

# Make the dataframe nice
df_final = df_init.T
df_final.columns = ['business_name'] + ls_col_names